# 🎵 Internship Project — Task 1  
## **Text-to-Music Generator using Sentiment and Mood Analysis**

### 👨‍💻 Submitted by:
**Name:** Prajwal Shinde  
**Internship Domain:** AI / NLP + Audio Generation  
**Mentor:** [Mentor292]  
**Submission:** Task 1 — Text Input & 20-Second Music Output  

---

### 🧠 **Objective**
The goal of this task is to develop a system that:
- Takes **text input** from the user  
- Analyzes its **sentiment** (positive, negative, or neutral)  
- Detects its **emotional tone/mood** (e.g., joy, sadness, anger, love, etc.)  
- Generates a **20-second piece of music** based on the detected mood and sentiment  

---

### ⚙️ **Process Overview**
1. **Input File (`input.txt`)** — contains user text lines.  
2. **Sentiment Detection** — using `cardiffnlp/twitter-roberta-base-sentiment-latest`.  
3. **Mood Detection** — using `bhadresh-savani/distilbert-base-uncased-emotion`.  
4. **Music Generation** — frequency-based waveform synthesis in NumPy.  
5. **Output Audio (`mood_sentiment_20s.wav`)** — 20-second unique audio clip.  

---

### 📂 **Final Submission Files**
- 📝 `input.txt` → Text input file  
- 🎧 `*_20s.wav` → 20-second audio file(s) generated from input  

---

### 💬 **Note**
All code and models are run on **Google Colab** for demonstration.  
Music files are generated dynamically using NumPy and saved in `.wav` format.


In [3]:
# ✅ STEP 1: Install dependencies
!pip install transformers torch numpy soundfile

# ✅ STEP 2: Import libraries
from transformers import pipeline
import numpy as np
import soundfile as sf

# ✅ STEP 3: Load your pretrained models
sentiment_model = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment-latest"
)

emotion_model = pipeline(
    "text-classification",
    model="bhadresh-savani/distilbert-base-uncased-emotion",
    return_all_scores=False
)

# ✅ STEP 4: Create the input text file
with open("input.txt", "w") as f:
    f.write("I am feeling peaceful and calm today.\n")
    f.write("I am very angry and frustrated right now.\n")
    f.write("Life feels neutral and simple today.\n")

print("✅ input.txt created successfully!")

# ✅ STEP 5: Define mood-based frequency patterns
mood_patterns = {
    'joy': ([523, 659, 784, 1046], [0.4, 0.4, 0.4, 0.4]),
    'anger': ([220, 220, 330, 440], [0.5, 0.5, 0.5, 0.5]),
    'sadness': ([220, 246, 261, 293], [0.6, 0.6, 0.6, 0.6]),
    'fear': ([440, 392, 349, 330], [0.5, 0.5, 0.5, 0.5]),
    'love': ([392, 440, 494, 523], [0.4, 0.4, 0.4, 0.4]),
    'surprise': ([523, 784, 988, 1319], [0.3, 0.3, 0.3, 0.3]),
    'neutral': ([440, 440, 440, 440], [0.5, 0.5, 0.5, 0.5])
}

# ✅ STEP 6: Define music generator (20-second version)
def generate_music(frequencies, durations, volume=0.5, total_duration=20):
    sr = 22050
    song = np.array([], dtype=np.float32)
    loop_time = sum(durations)
    repeat_count = int(total_duration // loop_time)

    for _ in range(repeat_count):
        for freq, dur in zip(frequencies, durations):
            freq *= np.random.uniform(0.95, 1.05)  # slight variation for realism
            t = np.linspace(0, dur, int(sr * dur), False)
            tone = volume * np.sin(2 * np.pi * freq * t)
            song = np.concatenate((song, tone))

    # Fill remaining time if needed
    if len(song) / sr < total_duration:
        remaining = total_duration - (len(song) / sr)
        t = np.linspace(0, remaining, int(sr * remaining), False)
        tone = volume * np.sin(2 * np.pi * frequencies[-1] * t)
        song = np.concatenate((song, tone))

    return song, sr

# ✅ STEP 7: Process each line from input.txt and generate music
with open("input.txt", "r") as f:
    lines = f.readlines()

for text in lines:
    text = text.strip()
    if not text:
        continue

    # Detect sentiment & mood
    sentiment = sentiment_model(text)[0]['label']
    mood = emotion_model(text)[0]['label']

    print(f"\n🎵 Text: {text}")
    print(f"💬 Sentiment: {sentiment}")
    print(f"🎭 Mood: {mood}")

    # Generate and save music
    freqs, durs = mood_patterns.get(mood, mood_patterns['neutral'])
    music, sr = generate_music(freqs, durs, total_duration=20)
    filename = f"{mood}_{sentiment}_20s.wav"
    sf.write(filename, music, sr)
    print(f"✅ Saved 20-second audio: {filename}")

print("\n🎯 All files generated successfully! Ready for submission.")

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
Device set to use cpu


✅ input.txt created successfully!

🎵 Text: I am feeling peaceful and calm today.
💬 Sentiment: positive
🎭 Mood: joy
✅ Saved 20-second audio: joy_positive_20s.wav

🎵 Text: I am very angry and frustrated right now.
💬 Sentiment: negative
🎭 Mood: anger
✅ Saved 20-second audio: anger_negative_20s.wav

🎵 Text: Life feels neutral and simple today.
💬 Sentiment: positive
🎭 Mood: joy
✅ Saved 20-second audio: joy_positive_20s.wav

🎯 All files generated successfully! Ready for submission.
